In [13]:
import pandas as pd
from pathlib import Path
import xarray as xr
from io import StringIO
import os

# https://github.com/dvignoles/rgispy
# mask creation
from rgispy.mask import get_point_mask_from_df, get_mask_ds

# sampling
from rgispy.sample import sample_ds

# utility
from rgispy.grid import count_non_nan

# sampled data use 
from rgispy.postprocess import join_sampled_files, georeference_sampled, normalize_sampled_files, get_sampled_df_byattr

In [15]:
# these are on asrc servers. 
DANIEL = Path(os.environ['DANIEL'])
proj_root = DANIEL.joinpath('projects/NREL_watersecurity')
mask_nc = proj_root.joinpath('analysis/03min/nc/CONUS_Masks_HydroSTN30_03min_Static.nc')
network_nc = proj_root.joinpath('analysis/03min/nc/CONUS_Network_HydroSTN30_03min_Static.nc')

The [xarray](https://xarray.pydata.org/en/stable/) library is used as interface with these netcdf.

In [7]:
mask  = xr.open_dataset(mask_nc)
network = xr.open_dataset(network_nc)

The Network netcdf file is the reference for attributes of the network such as CellIDS, Cell Area, and Stream Order.
Each dataarray in the mask netcdf is a subset of cellids. For instance, the `Dams` layer is subset of the `network['ID']` dataarray containing only cells with Dams. 

In [8]:
network

<xarray.Dataset>
Dimensions:         (lat: 563, lon: 1156)
Coordinates:
  * lat             (lat) float64 24.77 24.83 24.88 24.92 ... 52.78 52.83 52.88
  * lon             (lon) float64 -124.7 -124.7 -124.6 ... -67.07 -67.03 -66.97
    spatial_ref     int64 ...
Data variables:
    ID              (lat, lon) float64 ...
    ToCell          (lat, lon) float32 ...
    FromCell        (lat, lon) float32 ...
    Order           (lat, lon) float32 ...
    BasinID         (lat, lon) float32 ...
    BasinCells      (lat, lon) float64 ...
    Travel          (lat, lon) float32 ...
    CellArea        (lat, lon) float64 ...
    CellLength      (lat, lon) float32 ...
    SubbasinArea    (lat, lon) float64 ...
    SubbasinLength  (lat, lon) float64 ...
Attributes:
    WBM_network:    /asrc/ecr/balazs/GHAAS/RGISarchive2/CONUS/Network/HydroST...
    project:        DOE-NREL Water Security
    crs:            +init=epsg:4326
    creation_date:  2021-08-16 11:49:13.530937

In [9]:
mask

<xarray.Dataset>
Dimensions:                   (lat: 563, lon: 1156)
Coordinates:
  * lat                       (lat) float64 24.77 24.83 24.88 ... 52.83 52.88
  * lon                       (lon) float64 -124.7 -124.7 ... -67.03 -66.97
    spatial_ref               int64 ...
Data variables:
    ID                        (lat, lon) float64 ...
    Dams                      (lat, lon) float64 ...
    HydroPower                (lat, lon) float64 ...
    Gauges                    (lat, lon) float64 ...
    GMLC_PowerPlants          (lat, lon) float64 ...
    GMLC_PowerPlants_Buffer5  (lat, lon) float64 ...
    Verification_Dams         (lat, lon) float64 ...
    Basins                    (lat, lon) float32 ...
Attributes:
    WBM_network:                         /asrc/ecr/balazs/GHAAS/RGISarchive2/...
    project:                             DOE-NREL Water Security
    crs:                                 +init=epsg:4326
    creation_date:                       2021-10-20T11:41:34.652650
    How to use stored Attribute Tables:  Import as pandas dataframe using com...

Each dataarray in the Mask should have its attribute table embedded as an attribute. 

In [10]:
dams_ref = pd.read_csv(StringIO(mask['Dams'].attrs['Attribute Table']),sep='\t')
dams_ref.head()

,Unnamed: 0,OID_,ID,Name,SymbolFLD,RECORDID,DAM_NAME,OTHER_DAM_NAME,DAM_FORMER_NAME,NIDID,...,NonIrrDemand,NatInflow,ResidencyTime_yr,UpStreamCount,HydroPower,ManualEdit,HighResidency,AdjustedX,AdjustedY,CellID
0,0,1,1,SOO LOCKS,Symbol:001,32978,SOO LOCKS,SUPERIOR,NaN,MI00650,...,0.031,2984.069580,3.637820,10,1,-1,-2,-84.375000,46.525002,139118
1,1,2,2,HOOVER,Symbol:001,58571,HOOVER,BOULDER,NaN,NV10122,...,0.212,268.125031,4.410887,49,1,-1,-2,-114.724998,36.025002,234512
2,2,3,3,GLEN CANYON,Symbol:001,3918,GLEN CANYON,NaN,NaN,AZ10307,...,0.012,254.129745,4.598085,43,1,1,-1,-111.474998,36.974998,234599
3,3,4,4,GARRISON DAM,Symbol:001,52676,GARRISON DAM,LAKE SAKAKAWEA,NaN,ND00145,...,19.128,268.887085,3.782056,29,1,-1,-2,-101.425003,47.474998,607
4,4,5,5,OAHE DAM,Symbol:001,73327,OAHE DAM,LAKE OAHE,NaN,SD01095,...,0.031,301.206940,3.064584,41,1,1,-1,-100.375000,44.424999,513


In [33]:
# we'll create this
thermal_mask_nc = Path.home().joinpath('CONUS_Mask_ThermalStrat_HydroSTN30_03min_Static.nc')

## Mask Creation
Create a subset of our existing Dams mask with the 18 NIDIDs supplied

This doesn't need to be run again, but is here for your reference.

In [34]:
thermal18 = pd.read_csv(proj_root.joinpath('thermal_stratification', 'ThermalTestNIDID2.csv'))
thermal18.head()

,TestReservoirID,lon,lat,NIDID,Name
0,1,-98.1875,28.4375,TX04425,CHOKE CANYON
1,2,-85.9375,32.6875,AL01425,MARTIN
2,3,-93.9375,33.6875,AR00536,MILLWOOD DAM - OKAY LEVEE
3,4,-81.1875,34.0625,SC00224,SALUDA
4,5,-78.5625,35.9375,NC01713,FALLS LAKE DAM


In [35]:
thermal18_match = dams_ref[dams_ref['NIDID'].isin(thermal18.NIDID)]
print('Num matches: ', len(thermal18_match))

Num matches:  18


The functions below abstract away what is actually happening. `get_point_mask_from_df` returns a subset of `network['ID']` which is nan if the ID is not in `thermal18_match.CellID`.

In [36]:
thermal_mask_ds = get_mask_ds(network)

# The CellID from thermal_mask is 
thermal_mask = get_point_mask_from_df(thermal18_match, network)
thermal_mask_ds = thermal_mask_ds.assign(ThermalStrat=thermal_mask)

# check
count_non_nan(thermal_mask_ds['ThermalStrat'])

18

In [37]:
thermal_mask_ds.to_netcdf(thermal_mask_nc)

In [38]:
thermal_mask_ds.close()

## Sampling

In this step we use our new mask to sample the raw model outputs (datastreams).

The datastreams can be thought of as containing an array of grids, which each grid being a step in the time domain. So for a "daily" datastream, `CONUS_Output_RiverTemperature_TCfull+WBM20WTempDist19_03min_dTS2001.gds.gz`, there are 365 2d-arrays. Each 2d-array covers the entire model domain (CONUS 03min), in the same shape as the network. For the purposes of sampling we don't need the value at every grid cell. The mask we created is used to *mask* each output grid extracting just the cells we are interested in. 

The example below samples 1990-1992 for just a few variables. Modify to your needs. 

In [39]:
help(sample_ds)

Help on function sample_ds in module rgispy.sample:

sample_ds(mask_nc: pathlib.Path, file_in: Union[BinaryIO, pathlib.Path], mask_layers: List[str], output_dir: pathlib.Path, year: Optional[int], variable: str, time_step: str, csv_name: Union[str, pathlib.Path] = None, cell_area: numpy.ndarray = None) -> None
    Sample a datastream using a netcdf mask
    
    Args:
        mask_nc (Path): netcdf mask file
        file_in (Union[BinaryIO, Path]): datastream file object or pathlike
        mask_layers (List[str]): list of masks from mask_nc to sample with
        output_dir (Path): directory of output
        year (Optional[int]): year of datastream file
        variable (str): variable of datastream file (ie. Discharge, Temperature..)
        time_step (str): annual, monthly, daily, alt, or dlt
        csv_name (Union[str, Path]): Name of resulting sampled csv
        cell_area (Optional[np.ndarray]): Cell Area grid corresponding to mask. Needed for polygon masks to calculate weighte

In [40]:
# The SNL run. Contains gds datastream files. 
D_RUN = Path('/asrc/ecr/balazs/Projects/DOE-NREL_GMLC/WBMtemp/CONUS/Network_03min/TCfull+WBM20WTempDist19')

# may want to change output location
OUTPUT_DIR = Path.home().joinpath('CONUS_03min_TCfull+WBM20WTempDist19_SNL')

In [41]:
# Variables we'll sample. 
TO_SAMPLE = ['RiverTemperature', 'ReservoirInflow', 'ReservoirRelease']

`sample_ds` requires us to extract some information from the file name as input. This generator extracts the year and filters to a year range.

In [42]:
def get_wbm_output(output_dir, variable_str, year_start=1990, year_end=2020):
    """Assemble information for each datastream needed to call sample_ds"""
    for f in sorted(output_dir.glob("*_{}_*dTS*.gds.gz".format(variable_str))):
        year = int(f.name.split('.')[0].split('_')[-1][3:])
        if year >= year_start and year <= year_end:
            yield (f, year, variable_str, 'daily')

Notice we pass the mask netcdf we created `thermal_mask_nc`, and the mask layer we wish to use (ThermalStrat). 

If your mask netcdf contains multiple masks, you can pass multiple mask names here and sample them simultaneously. 

In [43]:
for variable_str in TO_SAMPLE:
    for f, year, _, time_step in get_wbm_output(D_RUN, variable_str, year_start=2000, year_end=2002):
        sample_ds(thermal_mask_nc, f, ['ThermalStrat',], OUTPUT_DIR, year, variable_str, time_step, )
        print(f.name)

CONUS_Output_RiverTemperature_TCfull+WBM20WTempDist19_03min_dTS2000.gds.gz
CONUS_Output_RiverTemperature_TCfull+WBM20WTempDist19_03min_dTS2001.gds.gz
CONUS_Output_RiverTemperature_TCfull+WBM20WTempDist19_03min_dTS2002.gds.gz
CONUS_Output_ReservoirInflow_TCfull+WBM20WTempDist19_03min_dTS2000.gds.gz
CONUS_Output_ReservoirInflow_TCfull+WBM20WTempDist19_03min_dTS2001.gds.gz
CONUS_Output_ReservoirInflow_TCfull+WBM20WTempDist19_03min_dTS2002.gds.gz
CONUS_Output_ReservoirRelease_TCfull+WBM20WTempDist19_03min_dTS2000.gds.gz
CONUS_Output_ReservoirRelease_TCfull+WBM20WTempDist19_03min_dTS2001.gds.gz
CONUS_Output_ReservoirRelease_TCfull+WBM20WTempDist19_03min_dTS2002.gds.gz


## Sampled Data Importing 
You can operate on the csvs outputted in the previous step however you like, but I set up these functions to conveniently read in data split across multiple csvs and perform some transformations / filtering.

run `help(<func>) on any rgispy function to get a description` ie `help(join_sampled_files)`

In [44]:
DAILY_SAMPLE = sorted(OUTPUT_DIR.rglob('*ReservoirRelease*'))

Lets read in our table of the 16 subset Dams. We'll need the attributes if we want to normalize or filter the results. 

In [45]:
thermal_mask_ds = xr.open_dataset(thermal_mask_nc)
thermalstrat_dams = pd.read_csv(StringIO(thermal_mask_ds['ThermalStrat'].attrs['Attribute Table']),sep='\t')
thermal_mask_ds.close()
thermalstrat_dams.head()

,Unnamed: 0,OID_,ID,Name,SymbolFLD,RECORDID,DAM_NAME,OTHER_DAM_NAME,DAM_FORMER_NAME,NIDID,...,NonIrrDemand,NatInflow,ResidencyTime_yr,UpStreamCount,HydroPower,ManualEdit,HighResidency,AdjustedX,AdjustedY,CellID
0,1,2,2,HOOVER,Symbol:001,58571,HOOVER,BOULDER,NaN,NV10122,...,0.212,268.125031,4.410887,49,1,-1,-2,-114.724998,36.025002,234512
1,3,4,4,GARRISON DAM,Symbol:001,52676,GARRISON DAM,LAKE SAKAKAWEA,NaN,ND00145,...,19.128,268.887085,3.782056,29,1,-1,-2,-101.425003,47.474998,607
2,4,5,5,OAHE DAM,Symbol:001,73327,OAHE DAM,LAKE OAHE,NaN,SD01095,...,0.031,301.206940,3.064584,41,1,1,-1,-100.375000,44.424999,513
3,5,6,6,FORT PECK DAM,Symbol:001,46077,FORT PECK DAM,FORT PECK LAKE,NaN,MT00025,...,0.014,95.041992,7.860360,19,1,-1,-2,-106.425003,47.974998,1358
4,51,52,52,SALUDA,Symbol:001,70108,SALUDA,LAKE MURRAY DAM; LAKE MURRAY (RES),NaN,SC00224,...,3.984,66.916458,1.285921,4,1,-1,-2,-81.224998,34.075001,324717


join up all selected csvs into 1 dataframe

In [46]:
df = join_sampled_files(DAILY_SAMPLE)
df.head()

,2000-01-01,2000-01-02,2000-01-03,2000-01-04,2000-01-05,2000-01-06,2000-01-07,2000-01-08,2000-01-09,2000-01-10,...,2002-12-22,2002-12-23,2002-12-24,2002-12-25,2002-12-26,2002-12-27,2002-12-28,2002-12-29,2002-12-30,2002-12-31
cellid,,,,,,,,,,,,,,,,,,,,,
286433,43.194096,43.194584,43.195683,43.196045,43.196087,43.196293,43.193730,43.195160,43.195057,43.194035,...,14.303910,14.304208,14.304035,14.304182,14.304668,14.304835,14.304404,14.304564,14.305242,14.304928
513,286.534520,286.534520,286.534520,286.534520,286.534520,286.534520,286.534520,286.534520,286.534520,286.534520,...,253.937640,253.937640,253.937640,253.937640,253.937640,253.937640,253.937640,253.937640,253.937640,253.937640
349476,48.639816,48.552845,48.596470,48.564026,48.573776,48.557728,48.487500,48.409996,48.438152,48.456837,...,52.192116,52.192116,52.192116,52.192116,52.192116,52.192116,52.192116,52.192116,52.192116,52.193320
5446,109.400430,109.389560,109.381130,109.386250,109.374010,109.406590,109.359140,109.368060,109.374130,109.374466,...,124.453450,124.453450,124.453450,124.453450,124.453450,124.453450,124.453450,124.453450,124.453450,124.491230
304742,-0.000069,-0.000069,-0.000069,-0.000069,-0.000069,-0.000069,0.000007,0.000076,0.000038,0.000047,...,0.000533,0.000592,0.000631,0.000583,0.000623,0.000614,0.000597,0.000584,0.000576,0.000574


add location of wbm cell

In [47]:
df_g = georeference_sampled(df, network)
df_g[['2000-01-01', '2001-01-01', 'longitude', 'latitude']].head()

,2000-01-01,2001-01-01,longitude,latitude
cellid,,,,
286433,43.194096,29.225620,-85.925003,32.674999
513,286.534520,294.033570,-100.375000,44.424999
349476,48.639816,43.588085,-122.675003,42.674999
5446,109.400430,65.510710,-93.925003,33.674999
304742,-0.000069,0.000150,-98.224998,28.475000


Normalize (dn_n.sampleid = thermalstrat_dams.ID)

In [48]:
df_n = normalize_sampled_files(DAILY_SAMPLE, 'ReservoirRelease', thermalstrat_dams)
df_n

reservoirrelease
sampleid date                        
2        2000-01-01      2.511592e+02
         2000-01-02      2.508180e+02
         2000-01-03      2.508824e+02
         2000-01-04      2.509496e+02
         2000-01-05      2.509088e+02
...                               ...
1081     2002-12-27      1.186312e-07
         2002-12-28      1.122661e-07
         2002-12-29      1.074390e-07
         2002-12-30      1.047467e-07
         2002-12-31      1.067369e-07

[19728 rows x 1 columns]

Filtering. Lets look at Hoover dam.

In [49]:
thermalstrat_dams.loc[thermalstrat_dams['DAM_NAME'] == 'HOOVER', ['ID', 'NIDID', 'CellID']]

,ID,NIDID,CellID
0,2,NV10122,234512


In [50]:
HOOVER = 'NV10122'

We can use this function to filter by *any* attribute of thermalstrat_dams. In this case we pass `NIDID` as our filter column. 

In [51]:
get_sampled_df_byattr(DAILY_SAMPLE, thermalstrat_dams, 'NIDID', HOOVER,)

,2000-01-01,2000-01-02,2000-01-03,2000-01-04,2000-01-05,2000-01-06,2000-01-07,2000-01-08,2000-01-09,2000-01-10,...,2002-12-22,2002-12-23,2002-12-24,2002-12-25,2002-12-26,2002-12-27,2002-12-28,2002-12-29,2002-12-30,2002-12-31
cellid,,,,,,,,,,,,,,,,,,,,,
234512,251.15924,250.81805,250.88235,250.94957,250.90883,250.92798,250.96326,251.00371,250.9877,250.99693,...,246.49443,246.50415,246.52835,246.47318,246.4703,246.49542,246.45883,246.46278,246.47853,246.45586


The filtering function also has options for stacking/normalizing like above. 

In [52]:
get_sampled_df_byattr(DAILY_SAMPLE, thermalstrat_dams, 'NIDID', HOOVER, stacked=True, variable='ReservoirRelease').head()

ReservoirRelease
cellid date                        
234512 2000-01-01         251.15924
       2000-01-02         250.81805
       2000-01-03         250.88235
       2000-01-04         250.94957
       2000-01-05         250.90883

In [53]:
get_sampled_df_byattr(DAILY_SAMPLE, thermalstrat_dams, 'NIDID', HOOVER,  normalize=True, stacked=True, variable='ReservoirRelease',).head()

reservoirrelease
sampleid date                        
2        2000-01-01         251.15924
         2000-01-02         250.81805
         2000-01-03         250.88235
         2000-01-04         250.94957
         2000-01-05         250.90883